In [1]:
# comparison of various conv1d models on the SPEECHCOMMANDS dataset
# classic / fgn trained from scratch / converted

In [2]:
%load_ext autoreload
%autoreload 2
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8.0, 6.0)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [7]:
# load dataset
(train_loader, val_loader, test_loader) = fgnh.SpeechCommands_Dataloaders(resample_rate = 8000,
                                                                          batch_size = 32,
                                                                          batchsize_for_val =32,
                                                                          num_workers=5, 
                                                                          pin_memory=True)

In [8]:
# functions to test models
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, loader):
    model.eval()
    correct = 0
    for data, target in loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

    print(f"Accuracy: {correct}/{len(loader.dataset)} ({100. * correct / len(loader.dataset):.0f}%)\n")

In [9]:
# define model classes

## classic model
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)
    
# FGN model    
class FGN_M5(nn.Module):
    
    # changes:
    # nn.Conv1d -> fgnl.FGN_Conv1d
    # added g to conv inputs and outputs
    # make sure you pass g through the same pooling steps as x
    
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.fgn_conv1 = fgnl.FGN_Conv1d(in_channels=n_input, out_channels=n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.fgn_conv2 = fgnl.FGN_Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.fgn_conv3 = fgnl.FGN_Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.fgn_conv4 = fgnl.FGN_Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)
        
        #TODO change to self.pool1d_fgn() for each pooling of Gs

    def forward(self, x):
        x, g = self.fgn_conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        g = self.pool1(g)
        x, g = self.fgn_conv2(x, g)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        g = self.pool2(g)
        x, g = self.fgn_conv3(x ,g)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        g = self.pool3(g)
        x, _ = self.fgn_conv4(x, g)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2)    

In [10]:
# load models
save_path = '../Experiments/sample_SPEECHCOMMANDS_models/'

classic_model_name= 'sample_classic_model_SPEECHCOMMANDS'
fgn_model_name = 'sample_FGN_model_SPEECHCOMMANDS'

In [11]:
classic_model = M5()
classic_model.load_state_dict(torch.load(save_path+classic_model_name+'_state_dict.pth'))
classic_model.to(device)

/home/felix/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:125: UserWarning: 
GeForce RTX 2080 Ti with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the GeForce RTX 2080 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=35, bias=True)
)

In [12]:
fgn_model = FGN_M5()
fgn_model.load_state_dict(torch.load(save_path+fgn_model_name+'_state_dict.pth'))
fgn_model.to(device)

FGN_M5(
  (fgn_conv1): FGN_Conv1d(
    (Conv1d): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  )
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv2): FGN_Conv1d(
    (Conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv3): FGN_Conv1d(
    (Conv1d): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  )
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv4): FGN_Conv1d(
    (Conv1d): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): Ma

In [13]:
# test models
test(classic_model, train_loader)
test(classic_model, val_loader)
test(classic_model, test_loader)

Accuracy: 76655/84843 (90%)

Accuracy: 8755/9981 (88%)

Accuracy: 9468/11005 (86%)



In [14]:
test(fgn_model, train_loader)
test(fgn_model, val_loader)
test(fgn_model, test_loader)

Accuracy: 73489/84843 (87%)

Accuracy: 8572/9981 (86%)

Accuracy: 9244/11005 (84%)



In [15]:
import copy

In [16]:
converted_model = copy.deepcopy(fgn_model)

In [17]:
test(converted_model, train_loader)
test(converted_model, val_loader)
test(converted_model, test_loader)

Accuracy: 73489/84843 (87%)

Accuracy: 8572/9981 (86%)

Accuracy: 9244/11005 (84%)



In [18]:
for classic_model_layer,fgn_model_layer in zip(classic_model.children(), converted_model.children()):

    if type(fgn_model_layer)==fgnl.FGN_Conv1d:
        print('converting conv layer')
        fgnl.convert_layer_conv1D_to_fgn(classic_model_layer,fgn_model_layer)
    else:
        print('transfering state_dicts')
        fgn_model_layer.load_state_dict(classic_model_layer.state_dict())
        

converting conv layer
transfering state_dicts
transfering state_dicts
converting conv layer
transfering state_dicts
transfering state_dicts
converting conv layer
transfering state_dicts
transfering state_dicts
converting conv layer
transfering state_dicts
transfering state_dicts
transfering state_dicts


In [19]:
# check that the converted model matches classic behavior
test(converted_model, train_loader)
test(converted_model, val_loader)
test(converted_model, test_loader)

Accuracy: 76655/84843 (90%)

Accuracy: 8755/9981 (88%)

Accuracy: 9468/11005 (86%)



In [20]:
# check that the original model hasn't changed
test(converted_model, train_loader)
test(converted_model, val_loader)
test(converted_model, test_loader)

Accuracy: 76655/84843 (90%)

Accuracy: 8755/9981 (88%)

Accuracy: 9468/11005 (86%)



In [21]:
# retrain a bit

In [22]:
import torch.optim as optim

In [23]:
import IPython.display as ipd
from tqdm.notebook import tqdm

In [24]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = F.nll_loss(output.squeeze(), target) + lmbda_sigma*fgnl.sigmas_loss(model, covar_type='sphere')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [25]:
fast_retrained_converted_model = copy.deepcopy(converted_model)

In [26]:
lmbda_sigma = 1e-5
optimizer = optim.Adam(fast_retrained_converted_model.parameters(), lr=0.001, weight_decay=0.0001)
# note LR is 10x smaller because the models have already beenm trained for 20 epochs

In [27]:
# quick retrain the converted model
log_interval = 20
n_epoch = 1

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
# transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(fast_retrained_converted_model, epoch, log_interval)
        test(fast_retrained_converted_model, test_loader)

Train Epoch: 1 [0/84843 (0%)]	Loss: 569473.437500
Train Epoch: 1 [640/84843 (1%)]	Loss: 0.628793
Train Epoch: 1 [1280/84843 (2%)]	Loss: 0.644300
Train Epoch: 1 [1920/84843 (2%)]	Loss: 0.780621
Train Epoch: 1 [2560/84843 (3%)]	Loss: 0.536376
Train Epoch: 1 [3200/84843 (4%)]	Loss: 0.837728
Train Epoch: 1 [3840/84843 (5%)]	Loss: 0.442479
Train Epoch: 1 [4480/84843 (5%)]	Loss: 0.655623
Train Epoch: 1 [5120/84843 (6%)]	Loss: 0.628965
Train Epoch: 1 [5760/84843 (7%)]	Loss: 0.740122
Train Epoch: 1 [6400/84843 (8%)]	Loss: 0.767567
Train Epoch: 1 [7040/84843 (8%)]	Loss: 0.576215
Train Epoch: 1 [7680/84843 (9%)]	Loss: 0.557323
Train Epoch: 1 [8320/84843 (10%)]	Loss: 0.729879
Train Epoch: 1 [8960/84843 (11%)]	Loss: 0.667062
Train Epoch: 1 [9600/84843 (11%)]	Loss: 0.607314
Train Epoch: 1 [10240/84843 (12%)]	Loss: 0.752884
Train Epoch: 1 [10880/84843 (13%)]	Loss: 0.613645
Train Epoch: 1 [11520/84843 (14%)]	Loss: 0.605123
Train Epoch: 1 [12160/84843 (14%)]	Loss: 0.660557
Train Epoch: 1 [12800/84843 

In [28]:
# check that the fast retrained converted model improved and is good.
test(fast_retrained_converted_model, train_loader)
test(fast_retrained_converted_model, val_loader)
test(fast_retrained_converted_model, test_loader)

Accuracy: 76121/84843 (90%)

Accuracy: 8687/9981 (87%)

Accuracy: 9422/11005 (86%)



In [29]:
# retrain longer

In [30]:
long_retrained_converted_model = copy.deepcopy(fast_retrained_converted_model)

In [31]:
optimizer = optim.Adam(long_retrained_converted_model.parameters(), lr=0.001, weight_decay=0.0001)

In [32]:
# longer retrain the converted model
log_interval = 20
n_epoch = 20

pbar_update = 1 / (len(train_loader) + len(test_loader))
losses = []

# The transform needs to live on the same device as the model and the data.
# transform = transform.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(long_retrained_converted_model, epoch, log_interval)
        test(long_retrained_converted_model, test_loader)


Train Epoch: 1 [0/84843 (0%)]	Loss: 0.759155
Train Epoch: 1 [640/84843 (1%)]	Loss: 0.252511
Train Epoch: 1 [1280/84843 (2%)]	Loss: 0.373392
Train Epoch: 1 [1920/84843 (2%)]	Loss: 0.307572
Train Epoch: 1 [2560/84843 (3%)]	Loss: 0.424764
Train Epoch: 1 [3200/84843 (4%)]	Loss: 0.406423
Train Epoch: 1 [3840/84843 (5%)]	Loss: 0.835204
Train Epoch: 1 [4480/84843 (5%)]	Loss: 0.179899
Train Epoch: 1 [5120/84843 (6%)]	Loss: 0.740609
Train Epoch: 1 [5760/84843 (7%)]	Loss: 0.429581
Train Epoch: 1 [6400/84843 (8%)]	Loss: 0.445819
Train Epoch: 1 [7040/84843 (8%)]	Loss: 0.259833
Train Epoch: 1 [7680/84843 (9%)]	Loss: 0.245413
Train Epoch: 1 [8320/84843 (10%)]	Loss: 0.399233
Train Epoch: 1 [8960/84843 (11%)]	Loss: 0.549836
Train Epoch: 1 [9600/84843 (11%)]	Loss: 0.439092
Train Epoch: 1 [10240/84843 (12%)]	Loss: 0.345250
Train Epoch: 1 [10880/84843 (13%)]	Loss: 0.379441
Train Epoch: 1 [11520/84843 (14%)]	Loss: 0.646546
Train Epoch: 1 [12160/84843 (14%)]	Loss: 0.451731
Train Epoch: 1 [12800/84843 (15%)

Train Epoch: 2 [20480/84843 (24%)]	Loss: 0.573640
Train Epoch: 2 [21120/84843 (25%)]	Loss: 0.506166
Train Epoch: 2 [21760/84843 (26%)]	Loss: 0.400668
Train Epoch: 2 [22400/84843 (26%)]	Loss: 0.271863
Train Epoch: 2 [23040/84843 (27%)]	Loss: 0.507545
Train Epoch: 2 [23680/84843 (28%)]	Loss: 0.479954
Train Epoch: 2 [24320/84843 (29%)]	Loss: 0.758438
Train Epoch: 2 [24960/84843 (29%)]	Loss: 0.554884
Train Epoch: 2 [25600/84843 (30%)]	Loss: 0.418424
Train Epoch: 2 [26240/84843 (31%)]	Loss: 0.438241
Train Epoch: 2 [26880/84843 (32%)]	Loss: 0.359236
Train Epoch: 2 [27520/84843 (32%)]	Loss: 0.364366
Train Epoch: 2 [28160/84843 (33%)]	Loss: 0.472508
Train Epoch: 2 [28800/84843 (34%)]	Loss: 0.366748
Train Epoch: 2 [29440/84843 (35%)]	Loss: 0.229097
Train Epoch: 2 [30080/84843 (35%)]	Loss: 0.290334
Train Epoch: 2 [30720/84843 (36%)]	Loss: 0.435716
Train Epoch: 2 [31360/84843 (37%)]	Loss: 0.635519
Train Epoch: 2 [32000/84843 (38%)]	Loss: 0.426257
Train Epoch: 2 [32640/84843 (38%)]	Loss: 0.307621


Train Epoch: 3 [40320/84843 (48%)]	Loss: 0.552607
Train Epoch: 3 [40960/84843 (48%)]	Loss: 0.238592
Train Epoch: 3 [41600/84843 (49%)]	Loss: 0.210868
Train Epoch: 3 [42240/84843 (50%)]	Loss: 0.151025
Train Epoch: 3 [42880/84843 (51%)]	Loss: 0.415208
Train Epoch: 3 [43520/84843 (51%)]	Loss: 0.817350
Train Epoch: 3 [44160/84843 (52%)]	Loss: 0.566665
Train Epoch: 3 [44800/84843 (53%)]	Loss: 0.403705
Train Epoch: 3 [45440/84843 (54%)]	Loss: 0.271976
Train Epoch: 3 [46080/84843 (54%)]	Loss: 0.087232
Train Epoch: 3 [46720/84843 (55%)]	Loss: 0.311925
Train Epoch: 3 [47360/84843 (56%)]	Loss: 0.270397
Train Epoch: 3 [48000/84843 (57%)]	Loss: 0.201671
Train Epoch: 3 [48640/84843 (57%)]	Loss: 0.222735
Train Epoch: 3 [49280/84843 (58%)]	Loss: 0.621256
Train Epoch: 3 [49920/84843 (59%)]	Loss: 0.268777
Train Epoch: 3 [50560/84843 (60%)]	Loss: 0.510050
Train Epoch: 3 [51200/84843 (60%)]	Loss: 0.362784
Train Epoch: 3 [51840/84843 (61%)]	Loss: 0.448192
Train Epoch: 3 [52480/84843 (62%)]	Loss: 0.253372


Train Epoch: 4 [60160/84843 (71%)]	Loss: 0.203678
Train Epoch: 4 [60800/84843 (72%)]	Loss: 0.092860
Train Epoch: 4 [61440/84843 (72%)]	Loss: 0.526929
Train Epoch: 4 [62080/84843 (73%)]	Loss: 0.325101
Train Epoch: 4 [62720/84843 (74%)]	Loss: 0.504524
Train Epoch: 4 [63360/84843 (75%)]	Loss: 0.274383
Train Epoch: 4 [64000/84843 (75%)]	Loss: 0.213330
Train Epoch: 4 [64640/84843 (76%)]	Loss: 0.740426
Train Epoch: 4 [65280/84843 (77%)]	Loss: 0.504172
Train Epoch: 4 [65920/84843 (78%)]	Loss: 0.307395
Train Epoch: 4 [66560/84843 (78%)]	Loss: 0.870459
Train Epoch: 4 [67200/84843 (79%)]	Loss: 0.477396
Train Epoch: 4 [67840/84843 (80%)]	Loss: 0.147742
Train Epoch: 4 [68480/84843 (81%)]	Loss: 0.362520
Train Epoch: 4 [69120/84843 (81%)]	Loss: 0.143349
Train Epoch: 4 [69760/84843 (82%)]	Loss: 0.605306
Train Epoch: 4 [70400/84843 (83%)]	Loss: 0.459675
Train Epoch: 4 [71040/84843 (84%)]	Loss: 0.517581
Train Epoch: 4 [71680/84843 (84%)]	Loss: 0.255985
Train Epoch: 4 [72320/84843 (85%)]	Loss: 0.497531


Train Epoch: 5 [80000/84843 (94%)]	Loss: 0.455690
Train Epoch: 5 [80640/84843 (95%)]	Loss: 0.466225
Train Epoch: 5 [81280/84843 (96%)]	Loss: 0.233212
Train Epoch: 5 [81920/84843 (97%)]	Loss: 0.463883
Train Epoch: 5 [82560/84843 (97%)]	Loss: 0.574283
Train Epoch: 5 [83200/84843 (98%)]	Loss: 0.421111
Train Epoch: 5 [83840/84843 (99%)]	Loss: 0.600136
Train Epoch: 5 [84480/84843 (100%)]	Loss: 0.280031
Accuracy: 9446/11005 (86%)

Train Epoch: 6 [0/84843 (0%)]	Loss: 0.273857
Train Epoch: 6 [640/84843 (1%)]	Loss: 0.268012
Train Epoch: 6 [1280/84843 (2%)]	Loss: 0.220747
Train Epoch: 6 [1920/84843 (2%)]	Loss: 0.362594
Train Epoch: 6 [2560/84843 (3%)]	Loss: 0.356912
Train Epoch: 6 [3200/84843 (4%)]	Loss: 0.502520
Train Epoch: 6 [3840/84843 (5%)]	Loss: 0.540419
Train Epoch: 6 [4480/84843 (5%)]	Loss: 0.330759
Train Epoch: 6 [5120/84843 (6%)]	Loss: 0.340926
Train Epoch: 6 [5760/84843 (7%)]	Loss: 0.224527
Train Epoch: 6 [6400/84843 (8%)]	Loss: 0.360593
Train Epoch: 6 [7040/84843 (8%)]	Loss: 0.345960

...

Train Epoch: 15 [60160/84843 (71%)]	Loss: 0.617751
Train Epoch: 15 [60800/84843 (72%)]	Loss: 0.341723
Train Epoch: 15 [61440/84843 (72%)]	Loss: 0.454355
Train Epoch: 15 [62080/84843 (73%)]	Loss: 0.786526
Train Epoch: 15 [62720/84843 (74%)]	Loss: 0.338125
Train Epoch: 15 [63360/84843 (75%)]	Loss: 0.466006
Train Epoch: 15 [64000/84843 (75%)]	Loss: 0.517246
Train Epoch: 15 [64640/84843 (76%)]	Loss: 0.462289
Train Epoch: 15 [65280/84843 (77%)]	Loss: 0.457802
Train Epoch: 15 [65920/84843 (78%)]	Loss: 0.278409
Train Epoch: 15 [66560/84843 (78%)]	Loss: 0.425982
Train Epoch: 15 [67200/84843 (79%)]	Loss: 0.289424
Train Epoch: 15 [67840/84843 (80%)]	Loss: 0.361098
Train Epoch: 15 [68480/84843 (81%)]	Loss: 0.392337
Train Epoch: 15 [69120/84843 (81%)]	Loss: 0.243713
Train Epoch: 15 [69760/84843 (82%)]	Loss: 0.406092
Train Epoch: 15 [70400/84843 (83%)]	Loss: 0.530720
Train Epoch: 15 [71040/84843 (84%)]	Loss: 0.453920
Train Epoch: 15 [71680/84843 (84%)]	Loss: 0.259531
Train Epoch: 15 [72320/84843 (8

Train Epoch: 16 [78080/84843 (92%)]	Loss: 0.477911
Train Epoch: 16 [78720/84843 (93%)]	Loss: 0.618130
Train Epoch: 16 [79360/84843 (94%)]	Loss: 0.423323
Train Epoch: 16 [80000/84843 (94%)]	Loss: 0.753285
Train Epoch: 16 [80640/84843 (95%)]	Loss: 0.222553
Train Epoch: 16 [81280/84843 (96%)]	Loss: 0.401161
Train Epoch: 16 [81920/84843 (97%)]	Loss: 0.159475
Train Epoch: 16 [82560/84843 (97%)]	Loss: 0.431981
Train Epoch: 16 [83200/84843 (98%)]	Loss: 0.320566
Train Epoch: 16 [83840/84843 (99%)]	Loss: 0.409659
Train Epoch: 16 [84480/84843 (100%)]	Loss: 0.332064
Accuracy: 9413/11005 (86%)

Train Epoch: 17 [0/84843 (0%)]	Loss: 0.183403
Train Epoch: 17 [640/84843 (1%)]	Loss: 0.317997
Train Epoch: 17 [1280/84843 (2%)]	Loss: 0.529951
Train Epoch: 17 [1920/84843 (2%)]	Loss: 0.504628
Train Epoch: 17 [2560/84843 (3%)]	Loss: 0.221326
Train Epoch: 17 [3200/84843 (4%)]	Loss: 0.186031
Train Epoch: 17 [3840/84843 (5%)]	Loss: 0.283224
Train Epoch: 17 [4480/84843 (5%)]	Loss: 0.209057
Train Epoch: 17 [5120/

Train Epoch: 18 [10880/84843 (13%)]	Loss: 0.567463
Train Epoch: 18 [11520/84843 (14%)]	Loss: 0.407993
Train Epoch: 18 [12160/84843 (14%)]	Loss: 0.279603
Train Epoch: 18 [12800/84843 (15%)]	Loss: 0.514688
Train Epoch: 18 [13440/84843 (16%)]	Loss: 0.226527
Train Epoch: 18 [14080/84843 (17%)]	Loss: 0.585587
Train Epoch: 18 [14720/84843 (17%)]	Loss: 0.253296
Train Epoch: 18 [15360/84843 (18%)]	Loss: 0.391144
Train Epoch: 18 [16000/84843 (19%)]	Loss: 0.128084
Train Epoch: 18 [16640/84843 (20%)]	Loss: 0.236773
Train Epoch: 18 [17280/84843 (20%)]	Loss: 0.391316
Train Epoch: 18 [17920/84843 (21%)]	Loss: 0.799641
Train Epoch: 18 [18560/84843 (22%)]	Loss: 0.115183
Train Epoch: 18 [19200/84843 (23%)]	Loss: 0.173912
Train Epoch: 18 [19840/84843 (23%)]	Loss: 0.564562
Train Epoch: 18 [20480/84843 (24%)]	Loss: 0.393002
Train Epoch: 18 [21120/84843 (25%)]	Loss: 0.278547
Train Epoch: 18 [21760/84843 (26%)]	Loss: 0.195222
Train Epoch: 18 [22400/84843 (26%)]	Loss: 0.347320
Train Epoch: 18 [23040/84843 (2

Train Epoch: 19 [28800/84843 (34%)]	Loss: 0.230917
Train Epoch: 19 [29440/84843 (35%)]	Loss: 0.570217
Train Epoch: 19 [30080/84843 (35%)]	Loss: 0.161679
Train Epoch: 19 [30720/84843 (36%)]	Loss: 0.416764
Train Epoch: 19 [31360/84843 (37%)]	Loss: 0.466276
Train Epoch: 19 [32000/84843 (38%)]	Loss: 0.225475
Train Epoch: 19 [32640/84843 (38%)]	Loss: 0.219609
Train Epoch: 19 [33280/84843 (39%)]	Loss: 0.315111
Train Epoch: 19 [33920/84843 (40%)]	Loss: 0.660868
Train Epoch: 19 [34560/84843 (41%)]	Loss: 0.190762
Train Epoch: 19 [35200/84843 (41%)]	Loss: 0.244937
Train Epoch: 19 [35840/84843 (42%)]	Loss: 0.518983
Train Epoch: 19 [36480/84843 (43%)]	Loss: 0.774919
Train Epoch: 19 [37120/84843 (44%)]	Loss: 0.283380
Train Epoch: 19 [37760/84843 (44%)]	Loss: 0.402310
Train Epoch: 19 [38400/84843 (45%)]	Loss: 0.186124
Train Epoch: 19 [39040/84843 (46%)]	Loss: 0.444957
Train Epoch: 19 [39680/84843 (47%)]	Loss: 0.537916
Train Epoch: 19 [40320/84843 (48%)]	Loss: 0.376470
Train Epoch: 19 [40960/84843 (4

Train Epoch: 20 [46720/84843 (55%)]	Loss: 0.522850
Train Epoch: 20 [47360/84843 (56%)]	Loss: 0.317932
Train Epoch: 20 [48000/84843 (57%)]	Loss: 0.714553
Train Epoch: 20 [48640/84843 (57%)]	Loss: 0.331549
Train Epoch: 20 [49280/84843 (58%)]	Loss: 0.338114
Train Epoch: 20 [49920/84843 (59%)]	Loss: 0.450678
Train Epoch: 20 [50560/84843 (60%)]	Loss: 0.327990
Train Epoch: 20 [51200/84843 (60%)]	Loss: 0.252806
Train Epoch: 20 [51840/84843 (61%)]	Loss: 0.303290
Train Epoch: 20 [52480/84843 (62%)]	Loss: 0.922283
Train Epoch: 20 [53120/84843 (63%)]	Loss: 0.398921
Train Epoch: 20 [53760/84843 (63%)]	Loss: 0.341955
Train Epoch: 20 [54400/84843 (64%)]	Loss: 0.525481
Train Epoch: 20 [55040/84843 (65%)]	Loss: 0.431597
Train Epoch: 20 [55680/84843 (66%)]	Loss: 0.780367
Train Epoch: 20 [56320/84843 (66%)]	Loss: 0.378198
Train Epoch: 20 [56960/84843 (67%)]	Loss: 0.292054
Train Epoch: 20 [57600/84843 (68%)]	Loss: 0.219422
Train Epoch: 20 [58240/84843 (69%)]	Loss: 0.466148
Train Epoch: 20 [58880/84843 (6

In [33]:
# check that the fast retrained converted model improved and is good.
test(long_retrained_converted_model, train_loader)
test(long_retrained_converted_model, val_loader)
test(long_retrained_converted_model, test_loader)

Accuracy: 77708/84843 (92%)

Accuracy: 8725/9981 (87%)

Accuracy: 9516/11005 (86%)



In [34]:
# # ### save models

# model_name = 'sample_FGN_converted_model_SPEECHCOMMANDS'
# save_path = '../Experiments/sample_SPEECHCOMMANDS_models/'
# print('Saving trained model {} in {}'.format(model_name, save_path))

# # save model converted_model
# torch.save(converted_model, save_path+model_name+'_full.pth')

# # save model weights
# torch.save(converted_model.state_dict(), save_path+model_name+'_state_dict.pth')

In [35]:
# model_name = 'sample_FGN_converted_fast_retrained_model_SPEECHCOMMANDS'

# print('Saving trained model {} in {}'.format(model_name, save_path))

# # save model converted_model
# torch.save(fast_retrained_converted_model, save_path+model_name+'_full.pth')

# # save model weights
# torch.save(fast_retrained_converted_model.state_dict(), save_path+model_name+'_state_dict.pth')

In [36]:
# model_name = 'sample_FGN_converted_long_retrained_model_SPEECHCOMMANDS'

# print('Saving trained model {} in {}'.format(model_name, save_path))

# # save model converted_model
# torch.save(long_retrained_converted_model, save_path+model_name+'_full.pth')

# # save model weights
# torch.save(long_retrained_converted_model.state_dict(), save_path+model_name+'_state_dict.pth')